# Concepts

## DataArray and Dataset slicing

In [ ]:
import numpy as np
import scipp as sc

x = sc.Variable(dims=['x'], values=[1,2,3,4])
da = sc.DataArray(data=x,
                  coords={'x':x},
                  masks={'x':sc.less(x, 2 * sc.units.one)})
ds = sc.Dataset({'a':da})

Consider a data array `da` and a dataset `ds` with an aligned coord and an aligned mask.
The following conditions must hold:

In [ ]:
assert 'x' in da['x', 0:1].coords # range slice preserves coord as aligned
assert 'x' in da['x', 0:1].masks # range slice preserves coord as aligned
assert 'x' in da['x', 0].attrs # todo, should be unaligned coord
#assert 'x' in da['x', 0].coords # todo, point slice preserves coord as unaligned
#assert 'x' in da['x', 0].masks # todo, point slice preserves masks as aligned

In [ ]:
assert ds['a']['x', 0:1] == ds['x', 0:1]['a']
assert ds['a']['x', 0] == ds['x', 0]['a']

In [ ]:
assert 'x' in ds['a'].coords
assert 'x' in ds['x', 0:1].coords
assert 'x' not in ds['x', 0].coords # cannot have unaligned coord in dataset
assert 'x' in ds['x', 0:1]['a'].coords
assert 'x' in ds['a']['x', 0].attrs # todo, should be unaligned coord
#assert 'x' in ds['x', 0]['a'].coords # todo

assert 'x' in ds['a'].masks
assert 'x' in ds['x', 0:1].masks
#assert 'x' in ds['x', 0].masks # todo, should stay aligned mask
assert 'x' in ds['x', 0:1]['a'].masks
#assert 'x' in ds['a']['x', 0].masks # todo, should stay aligned mask
#assert 'x' in ds['x', 0]['a'].masks # todo, should stay aligned mask

In operations, aligned coords are compared:

In [ ]:
try:
    ok = da['x', 0:1] + da['x', 1:2]
except:
    ok = False
assert not ok

Mismatching unaligned coords are dropped:

In [ ]:
#assert da + da['x', 0] == da + da['x', 0].data # todo

A missing unaligned coord is interpreted as mismatch to ensure that:

In [ ]:
a = da['x', 0]
b = da['x', 1]
c = da['x', 2]
assert a + (b + c) == (a + b) + c

Insertion order does not matter for unaligned coords:

In [ ]:
a = da.copy()
a.attrs['x'] = 1.0 * sc.units.m # todo, should be unaligned coord
b = da.copy()
b.attrs['x'] = 2.0 * sc.units.m # todo, should be unaligned coord
ds1 = sc.Dataset()
ds2 = sc.Dataset()
ds1['a'] = a
ds1['b'] = b
ds2['b'] = b
ds2['a'] = a
assert ds1 == ds2

Insert into dataset with mismatching unaligned coord drops unaligned:

In [ ]:
ds = sc.Dataset()
ds.coords['x'] = x['x', 0]
ds['a'] = da['x', 1] # todo should drop 'x' from 'a' (currently preserved since attrs are distinct from coords)
assert ds.coords['x'] == ds['a'].coords['x'] # shadowing should NOT be supported

Insertion with aligned and unaligned masks:

In [ ]:
ds = sc.Dataset()
ds.masks['x'] = sc.less(x, 2 * sc.units.one) # aligned mask
masked = da.copy()
masked.masks['x'] = sc.less(x, 1 * sc.units.one) # aligned mask
try:
    ds['a'] = masked # mismatching aligned mask
except:
    ok = True
else:
    ok = False
assert ok

#masked.masks['x'].align = False # todo, enable when unaligned masks supported
try:
    # Note difference to coords, where unaligned coord is dropped
    ds['a'] = masked # shadowing should NOT be supported
except:
    ok = True
else:
    ok = False
assert ok

ds = sc.Dataset()
masked = da.copy()
masked.masks['x'] = sc.less(x, 1 * sc.units.one) # aligned mask
#masked.masks['x'].align = False # todo, enable when unaligned masks supported
ds['a'] = masked
masked.masks['x'] = sc.less(x, 2 * sc.units.one) # aligned mask
#masked.masks['x'].align = False # todo, enable when unaligned masks supported
#ds['b'] = masked # todo, enable when unaligned masks supported
#assert 'x' not in ds.masks # todo, enable when unaligned masks supported
#assert ds['a'].masks['x'] != ds['b'].masks['x'] # todo, enable when unaligned masks supported

If there is no aligned coord it is preserved for all items:

In [ ]:
ds = sc.Dataset()
ds['a'] = da['x', 0]
# ds['b'] = da['x', 1] # todo
assert 'x' not in ds.coords
assert 'x' in ds['a'].attrs # todo, should be unaligned coord
#assert 'x' in ds['b'].attrs # todo, should be unaligned coord
try:
    ds.coords['x'] = x['x', 0] # would introduce shadowing
except:
    ok = True
else:
    ok = False
#assert ok # todo

In [ ]:
edges = sc.Variable(dims=['x'], values=[1,2,3,4,5])
da.coords['x'] = edges
assert sc.concatenate(da['x', :2], da['x', 2:], 'x') == da
#assert sc.concatenate(da['x', 0], da['x', 1], 'x') == da['x', 0:2] # todo, concat of unaligned should give aligned, at least for dimension-coords
#assert sc.concatenate(da['x', :-1], da['x', -1], 'x') == da # todo, attr should be unaligned coord and concate of aligned and unaligned should work
da_yx = sc.concatenate(da['x', :2], da['x', 2:], 'y') # create 2-D coord
assert da_yx.coords['x'] == sc.concatenate(da.coords['x']['x', :3], da.coords['x']['x', 2:], 'y')

2-D coords for a dimension prevent operations between slices that are not along that dimension:

In [ ]:
da_2d = sc.DataArray(
    data=sc.Variable(['y', 'x'], shape=[2, 2]),
    coords={
        'x':sc.Variable(['y', 'x'], values=np.array([[1, 2], [3, 4]])),
        'y':sc.Variable(['y'], values=[3, 4])})

da_2d['x', 0] + da_2d['x', 1] # Same as with 1-D coord: x-coord differs but not aligned due to slice.
try:
    # 'y' sliced, so 'x' coord is aligned and yields different values from slices of 2-D coord.
    da_2d['y', 0] + da_2d['y', 1] 
except RuntimeError:
    ok = False
else:
    ok = True
assert not ok